# Variational Autoencoder (VAE)

Useful links: 
* Original paper http://arxiv.org/abs/1312.6114
* Helpful videos explaining the topic 
   * https://www.youtube.com/watch?v=P78QYjWh5sM     
   * http://videolectures.net/deeplearning2015_courville_autoencoder_extension/?q=aaron%20courville

In this homework we will train an autoencoder to model images of faces. For this we take "Labeled Faces in the Wild" dataset (LFW) (http://vis-www.cs.umass.edu/lfw/), deep funneled version of it. (frontal view of all faces)


## Prepare the data

In [0]:
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
import torch
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import trange
from IPython import display

**Collab setting**

In [2]:
# if you're running in colab,
# 1. go to Runtime -> Change Runtimy Type -> GPU
# 2. uncomment this:
!wget https://raw.githubusercontent.com/yandexdataschool/Practical_DL/hw3_19/homework03/lfw_dataset.py -O lfw_dataset.py

--2019-04-28 20:09:57--  https://raw.githubusercontent.com/yandexdataschool/Practical_DL/hw3_19/homework03/lfw_dataset.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2734 (2.7K) [text/plain]
Saving to: ‘lfw_dataset.py’

lfw_dataset.py      100%[===================>]   2.67K  --.-KB/s    in 0s      

2019-04-28 20:09:57 (58.6 MB/s) - ‘lfw_dataset.py’ saved [2734/2734]



In [0]:
# The following line fetches you two datasets: images, usable for autoencoder training and attributes.
# Those attributes will be required for the final part of the assignment (applying smiles), so please keep them in mind
from lfw_dataset import fetch_lfw_dataset
data,attrs = fetch_lfw_dataset(dimx=36,dimy=36)


In [0]:
data = data/255
np.savez("real.npz", Pictures=data.reshape(data.shape[0], 36*36*3))

In [0]:
X_train = data[:10000].reshape((10000, -1))
print(X_train.shape)
X_val = data[10000:].reshape((-1, X_train.shape[1]))
print(X_val.shape)

image_h = data.shape[1]
image_w = data.shape[2]

For simplicity we want all values of the data to lie in the interval $[0,1]$:

In [0]:
X_train = np.float32(X_train)
X_val = np.float32(X_val)

In [0]:
def plot_gallery(images, h, w, n_row=3, n_col=6):
    """Helper function to plot a gallery of portraits"""
    plt.figure(figsize=(1.5 * n_col, 1.7 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[i].reshape((h, w, 3)), cmap=plt.cm.gray, vmin=-1, vmax=1, interpolation='nearest')
        plt.xticks(())
        plt.yticks(())

In [0]:
plot_gallery(X_train, image_h, image_w)

In [0]:
train = data_utils.TensorDataset(torch.Tensor(X_train), torch.zeros(X_train.shape[0],)) # pseudo labels needed to define TensorDataset
train_loader = data_utils.DataLoader(train, batch_size=100, shuffle=True)

val = data_utils.TensorDataset(torch.Tensor(X_val), torch.zeros(X_val.shape[0],))
val_loader = data_utils.DataLoader(val, batch_size=1, shuffle=False)

# Autoencoder

Why to use all this complicated formulaes and regularizations, what is the need for variational inference? To analyze the difference, let's first train just an autoencoder on the data:

<img src="https://github.com/dzisandy/Deep-Learning/blob/master/HW3/Autoencoder_structure.png?raw=1" alt="Autoencoder">

In [0]:
dimZ = 128 # Considering face reconstruction task, which size of representation seems reasonable?

# Define the decoder and encoder as networks with one hidden fc-layer
# (that means you will have 2 fc layers in each net)
# Use ReLU for hidden layers' activations
# GlorotUniform initialization for W
# Zero initialization for biases
# It's also convenient to put sigmoid activation on output layer to get nice normalized pics

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        #TODO 
        '''https://stackoverflow.com/questions/49433936/how-to-initialize-weights-in-pytorch'''
        def initialization(layer):
            if type(layer) == nn.Linear:
                torch.nn.init.xavier_uniform(layer.weight)
                layer.bias.data.fill_(0.)


        
        self.encoder = nn.Sequential()
        self.encoder.add_module('enc_fc_layer_1', nn.Linear(3888, 1024))
        self.encoder.add_module('bn_1', nn.BatchNorm1d(1024))
        self.encoder.add_module('relu_1', nn.ReLU())
        self.encoder.add_module('enc_fc_layer_2', nn.Linear(1024, dimZ))
        self.encoder.add_module('bn_2', nn.BatchNorm1d(dimZ))
        self.encoder.apply(initialization)
        
        
        self.decoder = nn.Sequential()
        self.decoder.add_module('dec_fc_layer_1', nn.Linear(dimZ, 1024))
        self.decoder.add_module('bn_1', nn.BatchNorm1d(1024))
        self.decoder.add_module('relu_1', nn.ReLU())
        self.decoder.add_module('dec_fc_layer_2', nn.Linear(1024, 3888))
        self.decoder.add_module('sigmoid', nn.Sigmoid())
        self.decoder.apply(initialization)

        
        
    def forward(self, x):
        
        #TODO
        
        latent_code = self.encoder(x)
        reconstruction = self.decoder(latent_code)
        
        return reconstruction, latent_code

In [0]:
# Create MSE loss function
criterion = torch.nn.MSELoss()

autoencoder = Autoencoder().cuda()

# Use Adam optimizer
optimizer = optim.Adam(autoencoder.parameters())

In [0]:
# Train your autoencoder
# Visualize progress in reconstruction and loss decay

def loss_cuda(model, criterion, X):
    X = Variable(X).cuda()
    encode, _ = model(X)
    return criterion(encode, X)
  
def train(model, n_epoch, criterion, optimizer, train_loader, val_loader, image_h, image_w):
    train_loss = []
    image_progress = []
    for epoch in trange(n_epoch):
        model.train()
        for X in train_loader:
            loss = loss_cuda(model, criterion, X[0])
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            train_loss.append(loss.data.cpu().numpy())
        model.eval()
        for X in val_loader:
            encode, _ = model(Variable(X[0]).cuda())
        image_progress.append(encode[0])
        display.clear_output(wait = True)
        
        #reconstruction
        plt.figure(figsize = (10,5))
        plt.subplot(1,2,1)
        plt.imshow(X[0].cpu().numpy().reshape((image_h, image_w, 3)))
        plt.title('Original image')
        plt.subplot(1,2,2)
        plt.imshow(encode.data.cpu().numpy().reshape((image_h, image_w, 3)))
        plt.title('Encoded image')
        plt.show()
        
        #loss decay
        plt.figure(figsize = (10,5))
        plt.plot(train_loss)
        plt.title('Train loss')
        plt.xlabel('Iteration')
        plt.ylabel('Loss')
        plt.show()
    return image_progress

        

In [0]:
image_progress = train(autoencoder, 2, criterion, optimizer, train_loader, val_loader, image_h, image_w)

In [0]:
# Examine the reconstructions
for j, data in enumerate(val_loader, 0):
    inp = Variable(data[0].cuda())
    pred, _ = autoencoder(inp)
    plot_gallery([data[0].numpy(), pred.data.cpu().numpy()], image_h, image_w, n_row=1, n_col=2)
    if (j >= 9):
        break

Reconstruction is not bad, right? 

# Sampling

In [0]:
for i, (putin, y) in enumerate(val_loader):
    if i == 2754:
        break
plt.imshow(putin.numpy().reshape((image_w, image_w, 3)))

In [0]:
plt.figure(figsize=(10, 12))
plt.suptitle('Twin farm')
for i in range(len(image_progress[:2])):
    plt.subplots_adjust(bottom=0.0, left=.1, right=.9, top=.50, hspace=.15)
    plt.subplot(6, 5, 5*(i//5) + i % 5 + 1)
    plt.imshow(image_progress[i].clamp(0,1).data.cpu().numpy().reshape(image_w, image_h, 3))
    plt.title('Epoch = {}'.format(i * 5 + 1))
    plt.axis('off')
plt.tight_layout()
plt.show()

Let's now sample several latent vectors and perform inference from $z$, reconstruct an image given some random $z$ representations.

In [0]:
z = (np.random.randn(25, dimZ)*0.5).astype('float32')
output = autoencoder.decoder(Variable(torch.from_numpy(z)).cuda()).clamp(0, 1)
plot_gallery(output.data.cpu().numpy(), image_h, image_w, n_row=5, n_col=5)

So, if we sample $z$ from normal, whould we eventually generate all possible faces? What do you think?

# Variational Autoencoder

Bayesian approach in deep learning considers everything in terms of distributions. Now our encoder generates not just a vector $z$ but posterior ditribution $q(z|x)$. In our case distribution $q$ is Gaussian distibution $N(\mu, \sigma)$ with parameters $\mu$, $\sigma$. Technically, the first difference is that you need to split bottleneck layer in two. One dense layer will generate vector $\mu$, and another will generate vector $\sigma$. Reparametrization trick should be implemented using the **gaussian_sampler** layer, that generates random vetor $\epsilon$ and returns $\mu+\sigma\epsilon \sim N(\mu, \sigma)$ .

Since our decoder is also a function that generates distribution, we need to do the same splitting for output layer. When testing the model we will look only on mean values, so one of the output will be actual autoencoder output.

In this homework we only ask for implementation of the simplest version of VAE - one $z$ sample per input. You can consider to sample several outputs from one input and average them.

In [0]:
dimZ = 128

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.pre_enc = nn.Sequential()
        self.pre_enc.add_module('enc_fc_layer_1', nn.Linear(3888, 1024))
        self.pre_enc.add_module('bn_1', nn.BatchNorm1d(1024))
        self.pre_enc.add_module('relu_1', nn.ReLU())

        self.encoder_mu = nn.Sequential()
        self.encoder_mu.add_module('enc_fc_layer_2', nn.Linear(1024, dimZ))
        self.encoder_mu.add_module('bn_2', nn.BatchNorm1d(dimZ))

        self.encoder_logsigma = nn.Sequential()
        self.encoder_logsigma.add_module('enc_fc_layer_3', nn.Linear(1024, dimZ))
        self.encoder_logsigma.add_module('bn_3', nn.BatchNorm1d(dimZ))

        self.pre_dec = nn.Sequential()
        self.pre_dec.add_module('bn_4', nn.BatchNorm1d(dimZ))
        self.pre_dec.add_module('dec_fc_layer_1', nn.Linear(dimZ, 1024))
        self.pre_dec.add_module('relu_2', nn.ReLU())

        self.dec_mu = nn.Linear(1024, 3888)
        self.dec_sigma = nn.Linear(1024, 3888)



    def gaussian_sampler(self, mu, logsigma):
        if self.training:
            std = logsigma.exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
            return mu
        
    def encode(self, x):
        x1 = self.pre_enc(x)
        latent_mu = self.encoder_mu(x1)
        latent_logsigma = self.encoder_logsigma(x1)/10
        return latent_mu, latent_logsigma
    
    def decode(self, z):
        z1 = self.pre_dec(z)
        reconstruction_mu = self.dec_mu(z1)
        reconstruction_logsigma = self.dec_sigma(z1)
        return reconstruction_mu, reconstruction_logsigma

    def forward(self, x):
        latent_mu, latent_logsigma = self.encode(x)
        reconstruction_mu, reconstruction_logsigma = self.decode(self.gaussian_sampler(latent_mu, latent_logsigma))
        return reconstruction_mu, reconstruction_logsigma, latent_mu, latent_logsigma

And the last, but not least! Place in the code where the most of the formulaes goes to - optimization objective. The objective for VAE has it's own name - variational lowerbound. And as for any lowerbound our intention is to maximize it. Here it is (for one sample $z$ per input $x$):

$$\mathcal{L} = -D_{KL}(q_{\phi}(z|x)||p_{\theta}(z)) + \log p_{\theta}(x|z)$$

Your next task is to implement two functions that compute KL-divergence and the second term - log-likelihood of an output. Here is some necessary math for your convenience:

$$D_{KL} = -\frac{1}{2}\sum_{i=1}^{dimZ}(1+log(\sigma_i^2)-\mu_i^2-\sigma_i^2)$$
$$\log p_{\theta}(x|z) = \sum_{i=1}^{dimX}\log p_{\theta}(x_i|z)=\sum_{i=1}^{dimX} \log \Big( \frac{1}{\sigma_i\sqrt{2\pi}}e^{-\frac{(\mu_I-x)^2}{2\sigma_i^2}} \Big)=...$$

Don't forget in the code that you are using $\log\sigma$ as variable. Explain, why not $\sigma$?

In [0]:
def KL_divergence(mu, logsigma):
    return -(1 + 2 * logsigma - mu**2 - torch.exp(2 * logsigma)).sum(dim=1).mean(dim=0) / 2

def log_likelihood(x, mu, logsigma):
    return (-logsigma - np.log(2 * np.pi) / 2 - (mu - x)**2 / (2 * torch.exp(2 * logsigma))).sum(dim=1).mean(dim=0)

def loss_vae(x, mu_gen, logsigma_gen, mu_z, logsigma_z):
    return 0.5 * KL_divergence(mu_z, logsigma_z) - log_likelihood(x, mu_gen, logsigma_gen)

And train the model:

In [0]:
# train your autoencoder
# visualize progress in reconstruction and loss decay
def train_vae(model, n_epoch, optimizer, train_loader, val_loader, image_h, image_w):
    train_loss = []
    for epoch in trange(n_epoch):
        val
        model.train()
        for X in train_loader:
            if epoch > 0:
                rec_mu, rec_logsigma, lat_mu, lat_logsigma = model(Variable(X[0]).cuda())
                loss = loss_vae(Variable(X[0]).cuda(), rec_mu, rec_logsigma, lat_mu, lat_logsigma)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                train_loss.append(loss.data.cpu().numpy())
            else:
                for param in model.parameters():
                    param.data.clamp_(-1/2, 1/2)
        model.eval()
        for X in val_loader:
            encode, _, _, _ = model(Variable(X[0]).cuda())
        display.clear_output(wait = True)
        
        plt.figure(figsize = (10,5))
        plt.subplot(1,2,1)
        plt.imshow(X[0].cpu().numpy().reshape((image_h, image_w, 3)))
        plt.title('Original image')
        plt.subplot(1,2,2)
        plt.imshow(encode.data.cpu().numpy().reshape((image_h, image_w, 3)))
        plt.title('Encoded image')
        plt.show()
        
        plt.figure(figsize = (10,5))
        plt.plot(train_loss)
        plt.title('Train loss')
        plt.xlabel('Iteration')
        plt.ylabel('Loss')
        plt.show()
        

In [0]:
vae_model = VAE().cuda()
optimizer_vae = optim.Adam(vae_model.parameters())
train_vae(vae_model, 100, optimizer_vae, train_loader, val_loader, image_h, image_w)

In [0]:
val_loader = data_utils.DataLoader(val, batch_size=1, shuffle=True)
vae_model.eval()
for j, data in enumerate(val_loader, 0):
    input = Variable(data[0].cuda())
    reconstruction_mu, _, _, _ = vae_model(input)
    plot_gallery([data[0].numpy(), reconstruction_mu.data.cpu().numpy()], image_h, image_w, n_row=1, n_col=2)
    if (j >= 9):
        break

And finally sample from VAE.

In [0]:
# TODO
# Sample some images from the learned distribution
# 1) Sample z ~ N(0,1)
# 2) Sample from N(decoder_mu(z), decoder_sigma(z))
z = torch.from_numpy(np.random.standard_normal((25, dimZ))).float()
vae_out, _ = vae_model.decode(Variable(z.cuda()))
plot_gallery(vae_out.data.cpu().numpy(), image_h, image_w, n_row = 5, n_col = 5)

Even if in practice you do not see the much difference between AE and VAE, or VAE is even worse, the little bayesian inside you should be jumping for joy right now. 

In VAE you can truly sample from image distribution $p(x)$, while in AE there is no easy and correct way to do it. 

## If you are going to do part 4 (presicion-recall score): 

In [0]:
# using your model, generate face set with len = len(data)
z = torch.from_numpy(np.random.standard_normal((13143, dimZ))).float()
answ, _ = vae_model.decode(Variable(z.cuda()))
answ = answ.cpu().detach().numpy()

# answ = <generated dataset>

## Save it ot npz

In [0]:
np.savez("vae.npz", Pictures=answ)

# Congrats! and Bonus

If you managed to tune your autoencoders to converge and learn something about the world, now it's time to make fun out of it. As you may have noticed, there are face attributes in dataset. We're interesting in "Smiling" column, but feel free to try others as well! Here is the first task:

1) Extract the "Smilling" attribute and create a two sets of images: 10 smiling faces and 10 non-smiling ones.

2) Compute latent representations for each image in "smiling" set and average those latent vectors. Do the same for "non-smiling" set. You have found **"vector representation"** of the "smile" and "no smile" attribute.

3) Compute the difference: "smile" vector minus "non-smile" vector.

3) Now check if **"feature arithmetics"** works. Sample a face without smile, encode it and add the diff from p. 3. Check if it works with both AE and VAE. 

In [0]:
from lfw_dataset import fetch_lfw_dataset
data,attrs = fetch_lfw_dataset(dimx=36,dimy=36)

In [0]:
def array(data,size):
    out = np.array(data)
    out = out.reshape((size,-1))
    out = out.astype('float32')
    out = out /255
    return out
  
def set_gen(data, attrs, length, sign, value):
    out = []
    for i in range(len(attrs['Smiling'])):
        if sign == 0:
            if attrs['Smiling'][i] > value:
                out.append(data[i])
            if len(out) == length:
                return out
        else:
            if attrs['Smiling'][i] < value:
                out.append(data[i])
            if len(out) == length:
                return out

## 1. Data generation

In [0]:
smiling = set_gen(data, attrs, length = 10, sign = 0, value =  2.5)
nonsmiling = set_gen(data,attrs, length = 10, sign = 1, value = -2.5)

x_sm = array(smiling,10)
x_nsm = array(nonsmiling,10)

## 2. Finding diff for VAE

In [0]:
vae_model.eval()

zsm, _ = vae_model.encode(Variable(torch.from_numpy(x_sm).float().cuda()))
znsm, _ = vae_model.encode(Variable(torch.from_numpy(x_nsm).float().cuda()))
diff_vae = torch.mean(zsm,dim=0) - torch.mean(znsm,dim=0)

## 3. Generating test set

In [0]:
nsm_test = set_gen(data,attrs, length = 20, sign = 1, value = -2.)
x_nsm_test = array(nsm_test,20)

## 4. Testing VAE

In [0]:
plot_gallery(x_nsm_test, image_h, image_w, n_row=4, n_col=5)

In [0]:
znsm_vae_test, _ = vae_model.encode(Variable(torch.from_numpy(x_nsm_test).float().cuda()))
smile_reconst_vae, _ = vae_model.decode(znsm_vae_test + diff_vae)
plot_gallery(smile_reconst_vae.data.cpu().numpy(), image_h, image_w, n_row=4, n_col=5)

## 5. Finding diff for AE

In [0]:
autoencoder.eval()

zsm_ae = autoencoder.encoder(Variable(torch.from_numpy(x_sm).float().cuda()));
znsm_ae = autoencoder.encoder(Variable(torch.from_numpy(x_nsm).float().cuda()));
diff_ae = torch.mean(zsm_ae,dim=0)-torch.mean(znsm_ae,dim=0);

## 6. Testing AE

In [0]:
plot_gallery(x_nsm_test, image_h, image_w, n_row=4, n_col=5)

In [0]:
znsm_ae_test = autoencoder.encoder(Variable(torch.from_numpy(x_nsm_test).float().cuda()));

In [0]:
smile_reconst_ae = autoencoder.decoder(znsm_ae_test + diff_ae)

In [0]:
plot_gallery(smile_reconst_ae.data.cpu().numpy(), image_h, image_w, n_row=4, n_col=5)